# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/neSFl/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298273872042                   -0.85    5.0         
  2   -8.300162575829       -2.72       -1.25    1.2    168ms
  3   -8.300430738492       -3.57       -1.89    2.1    190ms
  4   -8.300461085854       -4.52       -2.76    2.6    250ms
  5   -8.300463888948       -5.55       -3.06    2.6    209ms
  6   -8.300464215756       -6.49       -3.21    4.4    228ms
  7   -8.300464417346       -6.70       -3.35    8.2    358ms
  8   -8.300464526519       -6.96       -3.48    2.1    209ms
  9   -8.300464614448       -7.06       -3.70    1.6    241ms
 10   -8.300464621129       -8.18       -3.76    1.1    172ms
 11   -8.300464638866       -7.75       -4.00    2.1    230ms
 12   -8.300464643878       -8.30      

In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67397743935                   -0.70    6.5         
  2   -16.67868835286       -2.33       -1.14    1.5    436ms
  3   -16.67919603894       -3.29       -1.86    2.0    482ms
  4   -16.67927761417       -4.09       -2.75    4.5    552ms
  5   -16.67928598685       -5.08       -3.17    4.6    662ms
  6   -16.67928620149       -6.67       -3.46    2.5    485ms
  7   -16.67928621814       -7.78       -3.93    2.0    418ms
  8   -16.67928622146       -8.48       -4.57    2.5    492ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32498835319                   -0.56    6.4         
  2   -33.33258557804       -2.12       -1.00    1.0    1.50s
  3   -33.33408325442       -2.82       -1.73    4.2    1.91s
  4   -33.33428508950       -3.70       -2.65    2.4    1.64s
  5   -33.33574897359       -2.83       -2.30    8.1    2.50s
  6   -33.33694239002       -2.92       -2.51    4.4    1.64s
  7   -33.33694278746       -6.40       -2.54    2.4    1.54s
  8   -33.33688398823   +   -4.23       -2.47    1.9    1.51s
  9   -33.33686440390   +   -4.71       -2.40    3.9    1.69s
 10   -33.33686287250   +   -5.81       -2.40    1.0    1.39s
 11   -33.33687239848       -5.02       -2.42    1.0    1.36s
 12   -33.33687549580       -5.51      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298547337031                   -0.85    5.1         
  2   -8.300281807947       -2.76       -1.59    1.4    128ms
  3   -8.300438540722       -3.80       -2.68    2.9    164ms
  4   -8.300438433689   +   -6.97       -2.54    7.5    319ms
  5   -8.300464282396       -4.59       -3.33    1.1    143ms
  6   -8.300464589634       -6.51       -3.77    2.1    182ms
  7   -8.300464639846       -7.30       -4.32    1.2    137ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32573182173                   -0.56    6.8         
  2   -33.31496462679   +   -1.97       -1.27    1.0    1.72s
  3   -15.51196823316   +    1.25       -0.44    6.4    2.71s
  4   -33.32481627155        1.25       -1.78    5.6    2.50s
  5   -32.88762572157   +   -0.36       -1.16    4.4    2.36s
  6   -31.73104602208   +    0.06       -0.95    4.5    2.28s
  7   -33.33186098161        0.20       -2.02    4.1    2.06s
  8   -33.32572106417   +   -2.21       -1.85    2.5    1.79s
  9   -33.33646504605       -1.97       -2.36    2.6    1.73s
 10   -33.33668517886       -3.66       -2.80    1.9    1.55s
 11   -33.33692415470       -3.62       -3.24    3.0    1.75s
 12   -33.33693611387       -4.92      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.